In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import pytz

In [2]:
# Define the observer's location
lat = 51.380001   # bath
lon = -2.360000
alt = 0          # assume sea level

In [3]:
# Load the data from the CSV file using pandas
data = pd.read_csv('landolt_north.csv')

In [4]:
# Define the time for which to calculate the azimuth and elevation
now = dt.datetime.now(pytz.utc)   # use UTC time
# now = dt.datetime(2023, 3, 9, 18, 0, 0, tzinfo=pytz.utc)  # example future time

In [5]:
# Calculate the Local Sidereal Time (LST)
gst = (now.replace(hour=0, minute=0, second=0, microsecond=0, tzinfo=pytz.utc) -
       dt.datetime(2000, 1, 1, 12, 0, 0, tzinfo=pytz.utc)).total_seconds() * (360/86400)
lst = (gst + lon*15) % 360

In [6]:
# Calculate the Hour Angle (HA) and Altitude (Alt) for each object
data['HA'] = np.radians(lst - data['ra'])

data['Alt'] = np.arcsin(np.sin(np.radians(lat))*np.sin(np.radians(data['dec'])) +
                        np.cos(np.radians(lat))*np.cos(np.radians(data['dec']))*np.cos(data['HA']))

data['Az'] = np.degrees(np.arccos((np.sin(np.radians(data['dec'])) -
                        np.sin(data['Alt'])*np.sin(np.radians(lat))) /
                        (np.cos(data['Alt'])*np.cos(np.radians(lat)))))

In [7]:
print(data.columns)

Index(['oid', 'Identifier', 'ra', 'dec', 'HA', 'Alt', 'Az'], dtype='object')


In [8]:
data['Az'] = np.degrees(np.arccos((np.sin(np.radians(data['dec'])) -
                        np.sin(data['Alt'])*np.sin(np.radians(lat))) /
                        (np.cos(data['Alt'])*np.cos(np.radians(lat)))))

In [9]:
# Adjust the Azimuth for objects east of South (Az > 180)
data.loc[data['HA'] > 0, 'Az'] = 360 - data['Az']

In [10]:
# Convert Altitude to Elevation
data['El'] = 90 - np.degrees(data['Alt'])

In [11]:
# Add the new columns with appropriate column titles
data.rename(columns={'RA': 'Right Ascension', 'Dec': 'Declination', 'Az': 'Azimuth', 'El': 'Elevation'}, inplace=True)

In [12]:
# Save the results to a new CSV file
data.to_csv('results.csv', index=False)